In [7]:
import cv2
import numpy as np
import time
from ultralytics import YOLO
import serial  # For USB communication

# Load the trained YOLO model
model = YOLO('best.pt')  # Path to your trained model

# Load the corner and drop points
corner_points = np.load('corner_points.npz')['corner_points']
drop_points = np.load('drop_points.npz', allow_pickle=True)['drop_points'].item()

# USB connection parameters
usb_port = 'COM8'  # Update this to your USB port
baud_rate = 9600

def detect_battery(frame):
    results = model.predict(frame)
    detections = []
    
    # Iterate through each result
    for result in results:
        if result.boxes is None:
            continue  # If there are no boxes, continue to the next result

        for detection in result.boxes:
            xywh = detection.xywh
            if len(xywh) != 4:
                print(f"Unexpected format in xywh: {xywh}")
                continue
            
            x, y, w, h = xywh[0], xywh[1], xywh[2], xywh[3]
            cls = detection.cls
            battery_type = ''
            
            if cls == 0:
                battery_type = 'AA'
            elif cls == 1:
                battery_type = '9V'
            elif cls == 2:
                battery_type = 'D'
            
            detections.append({
                'battery_type': battery_type,
                'coordinates': (x, y)
            })
    
    return detections

def get_height_offset(battery_type):
    if battery_type == 'AA':
        return 1.0
    elif battery_type == '9V':
        return 2.0
    elif battery_type == 'D':
        return 4.0
    else:
        return 0.0

def connect_to_robot():
    global ser
    ser = serial.Serial(usb_port, baud_rate, timeout=1)
    print(f"Connected to robot via USB at {usb_port}")

def move_to_position(x, y, z):
    command = f"MOVE X{x} Y{y} Z{z}\n"
    ser.write(command.encode())

def electromagnet(state):
    command = f"ELECTROMAGNET {state}\n"
    ser.write(command.encode())

def sort_battery(detection):
    battery_type = detection['battery_type']
    x, y = detection['coordinates']
    z_offset = get_height_offset(battery_type)
    move_to_position(x, y, 10 + z_offset)
    time.sleep(3)
    electromagnet("ON")
    time.sleep(3)
    drop_x, drop_y, drop_z = drop_points[battery_type]
    move_to_position(drop_x, drop_y, drop_z)
    time.sleep(3)
    electromagnet("OFF")
    time.sleep(3)

def main():
    connect_to_robot()
    cap = cv2.VideoCapture(1)  # Adjust depending on your camera

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        detections = detect_battery(frame)
        
        for detection in detections:
            x, y = detection['coordinates']
            battery_type = detection['battery_type']
            cv2.rectangle(frame, (int(x - 15), int(y - 15)), (int(x + 15), int(y + 15)), (0, 255, 0), 2)
            cv2.putText(frame, battery_type, (int(x), int(y - 20)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        cv2.imshow("Battery Detection", frame)
        
        for detection in detections:
            sort_battery(detection)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    ser.close()

if __name__ == "__main__":
    main()


SerialException: could not open port 'COM8': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)

In [8]:
import serial.tools.list_ports

ports = list(serial.tools.list_ports.comports())
for port in ports:
    print(port.device)


In [17]:
import serial

try:
    ser = serial.Serial('COM8', 9600, timeout=1)  # Adjust baud rate and timeout as needed
    print(f"Successfully connected to {ser.portstr}")
    ser.close()  # Close the port after testing
except serial.SerialException as e:
    print(f"Error: {e}")


Error: could not open port 'COM8': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)
